## Psevdo naključni generator

In [4]:
import secrets  # Za generiranje naključnih števil
import hashlib  # Za kriptografske hash funkcije
from typing import Callable # Za tipizacijo funkcij

### XOR

In [5]:
def xor_bytes(s1: bytes, s2: bytes) -> bytes:
    """Izvede operacijo XOR med podanima seznamoma bajtov in vrne seznam bajtov"""
    return bytes(x ^ y for x, y in zip(s1, s2))

## Ponavljanje semena do n znakov

In [6]:
def G_repeat(seed: bytes, n: int) -> bytes:
    g_key = bytearray()
    for i in range(n):
        g_key.append(seed[i % len(seed)])
    return bytes(g_key)

## Generatorji

### LCG32

In [7]:
def G_lcg32(seed: bytes, n: int) -> bytes:
    a = 1664525
    c = 1013904223
    m = 2**32
    
    s = int.from_bytes(seed, 'big')
    g_key = bytearray()
    
    while len(g_key) < n:
        s = (a * s + c) % m
        g_key.extend(s.to_bytes(4, 'big'))
    
    return bytes(g_key[:n])

### G( SHA-256 ) Števčni način

In [8]:
def G_sha256(seed: bytes, n: int) -> bytes:
    res = bytearray()
    i = 0
    while len(res) < n:
        # Uporabimo SHA-256 za generiranje pseudonaključnih bajtov
        # Mozne alternative: hashlib.md5, hashlib.sha1, hashlib.sha512
        res += hashlib.sha256(i.to_bytes(8, "big") + seed).digest()
        i += 1
    return bytes(res)[:n]

## Statistični testi (Napad na generator)
$$A: \{0, 1\}^n \mapsto \{0, 1\}$$

### Empirično merjenje prednosti

$$ \text{Adv}_\text{PRG}[A, G] \approx | \underset{k \overset{R}{\leftarrow} \mathcal{K}}{\text{Pr}}[A(G(k))=1] - \underset{r \overset{R}{\leftarrow} \{0, 1\}^n}{\text{Pr}}[A(r)=1]|$$

In [9]:
# A je napadalec, ki sprejme bajt niz dolžine n in vrne 0 ali 1
# G je PRG, ki sprejme seme in dolžino n ter vrne bajt niz dolžine n
# n je dolžina izhodnega niza
# trials je število ponovitev za oceno verjetnosti

def adv_prg(A: Callable[[bytes], int],
            G: Callable[[bytes, int], bytes],
            n: int,
            trials: int = 400) -> float:
    # Merimo PRG
    cnt_G = 0
    for _ in range(trials):
        seed = secrets.token_bytes(16) # izberemo pravo naključno seme
        x = G(seed, n) # vzorčimo PRG
        cnt_G += A(x) # prištejemo 1, če test vrne 1, sicer 0
    pG = cnt_G / trials
    
    # Merimo na pravih naključnih vrednostih
    cnt_R = 0
    for _ in range(trials):
        r = secrets.token_bytes(n) # vzorčimo pravo naključno vrednost
        cnt_R += A(r) # prištejemo rezultat testa
    pR = cnt_R / trials
    return abs(pG - pR)

### Mobilni razlikovalec

In [10]:
import math
# Preprost napadalec, ki preveri enakomernost porazdelitve bitov
# Vrne 1, če je porazdelitev bitov dovolj neuravnotežena, sicer 0
# Prejmete bajt niz, ki predstavlja zaporedje 16/32-bitnih števil

def A_monobit(data: bytes) -> int:
    one = sum(b.bit_count() for b in data)
    zero = len(data) * 8 - one
    return 1 if abs(one - zero) < 10* math.sqrt(len(data)*8) else 0

### LCG raziskovalec
$X_{i+1} = (a\cdot X_i + c) \bmod 2^w$

$X_{i+1} \bmod 2 = (a \cdot X_i + c) \bmod 2$

- $a \bmod 2 = 1$ in
- $c \bmod 2 = 1$

$X_{i+1} \bmod 2 = (X_i \bmod 2) + 1 \bmod 2$

In [11]:
# Napadalec, ki preveri, ali se parnost zaporednih 16/32-bitnih števil izmenjuje
# Vrne 1, če se kdaj zaporedni števili ujemata v parnosti, sicer 0
# Prejmete bajt niz, ki predstavlja zaporedje 16/32-bitnih števil

def A_alternate_even_odd(data: bytes) -> int:
    stevila = []
    for i in range(0, len(data) - 3, 4):
        stevilo = int.from_bytes(data[i:i+4], byteorder='big')
        stevila.append(stevilo)

    for i in range(len(stevila) - 1):
        n_lsb = stevila[i] % 2
        c_lsb = stevila[i + 1] % 2

        if n_lsb == c_lsb:
            return 1 
    return 0

## TOKOVNA ŠIFRA

In [12]:
def sc_enc(key: bytes, pt: bytes, prg=G_lcg32) -> bytes:
    keystream = prg(key, len(pt))
    ct = xor_bytes(pt, keystream)
    return ct

def sc_dec(key: bytes, ct: bytes, prg=G_lcg32) -> bytes:
    key_s = prg(key, len(ct))
    pt = xor_bytes(ct, key_s)
    return pt

sc_enc(b'my_key', b'hello world!')
sc_dec(b'my_key', bytes.fromhex('986900b165276d744dcce380'), G_sha256)

b'hello world!'

## Napadi na tokovno šifro

**LCG generator:**  

- $a = 1 664 525$
- $c = 1 013 904 223$
- $m = 2^{32}$

In [ ]:
# Preverimo ranjivosti:
def preveri_PRG(ct: bytes):
    print(f"Ponavljanje zaporedja sod, lih: {A_alternate_even_odd(ct)}")
    print(f"Primerjanje ničel in enic: {A_monobit(ct)}")

In [ ]:
# Nadaljujte tu
# Xn+1 = ( aXn + c ) mod m
partKey = b"POST /login HTTP/1.1\r\n"   # Tku vemo da ta oblika sporočila zgleda

def napadanje_prg(ct: bytes, partKey: bytes):
    seed = bytearray([ct[i] ^ partKey[i] for i in range(len(partKey))]) # Key = ct XOR pt
    X0 = seed[0:4]  # Vzamemo prve 4 byte (seed) in na podlagi njih lahko izračunamo za naslednje 4, ...
    pt = sc_dec(X0, ct[4:], G_lcg32) # Dešifriramo
    print(pt.decode('utf-8'))